# 닥터풋헬스 스마트스토어 고객 리뷰 수집
- 링크: https://smartstore.naver.com/foothealth/products/4868868795
- 수집할 데이터 요소: 
    1. 평점 (Rating): `rating`
    2. 아이디 (User ID): `user_id`
    3. 작성날짜 (Review Date): `review_date`
    4. 제품명 (Product Name): `product_name`
    5. 간단리뷰 (Brief Review): `brief_review`
    6. 리뷰유형 (Review Type): `review_type`
        - 이 변수는 '한달사용기', '재구매', 'BEST' 등의 값을 포함할 수 있다.
    7. 리뷰내용 (Review Content): `review_content`
    8. 추천수 (Number of Likes/Recommendations): `recommendation_count`

In [1]:
# -*- coding: utf-8 -*-
from datetime import datetime
import pandas as pd
import numpy as np
import time, random
import re
from tqdm.notebook import trange, tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
# Chrome Webdriver 불러오기
def chromeWebdriver():
    # 크롬 드라이버 자동 설치
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    
    # 드라이버 옵션 설정
    options = Options()
    options.add_experimental_option('detach', True) # 브라우저 바로 닫힘 방지
    options.add_experimental_option('excludeSwitches', ['enable-logging']) # 불필요한 메시지 제거
#     options.add_argument('--headless') # 백그라운드 작업
    options.add_argument('--no-sandbox') # Bypass OS security model

    driver = webdriver.Chrome(service=chrome_service, options=options)
    
    return driver

In [50]:
# 스크래핑할 스마트스토어 주소 불러오기
url = "https://smartstore.naver.com/foothealth/products/4868868795"
driver = chromeWebdriver()
driver.get(url)

In [51]:
SCROLL_PAUSE_SEC = 1

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 높이 초기화
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height: # 새로 업로드되는 페이지가 없다면
        break
    last_height = new_height

In [52]:
# 리뷰 버튼 클릭
driver.find_element(By.XPATH, r'//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]').click()
time.sleep(1)
# 최신순 정렬 클릭
driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[2]').click()
time.sleep(1)

# 리뷰 개수로 페이지 수 계산
num_reviews = driver.find_element(By.XPATH, r'//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a/strong').text 
num_reviews = int(num_reviews.replace(",", ""))
max_page = int(num_reviews//20 + 1)
print('리뷰 수: ',num_reviews,', 페이지 수: ', max_page)

리뷰 수:  6380 , 페이지 수:  320


In [53]:
# 각 크롤링 결과 저장하기 위한 리스트 선언 
rating = []
user_id = []
review_date = []
product_info = []
review_text = []
recommendation_count = []
image_yn = []

In [54]:
# 페이지를 처음부터 끝까지, (20년도까지) 이동하면서 데이터 수집
page = 1
date = '2024.01.01'

while (page <= max_page) & ('20.01.01' <= date) :
    lis = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul').find_elements(By.TAG_NAME, 'li') 
    print('● 현재 페이지:', page, ', 해당 페이지 리뷰수:', len(lis))

    for i in trange(1, len(lis) + 1):
        try: 
            rating.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em').text)
            driver.implicitly_wait(1)
        except:
            rating.append(np.nan)
        # print(rating)
        try:    
            user_id.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/strong').text)
            driver.implicitly_wait(1)
        except:
            user_id.append(np.nan)
        # print(user_id)
        try: 
            date = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/span').text
            review_date.append(date)
            driver.implicitly_wait(1)
        except:
            review_date.append(np.nan)
        # print(review_date)
        try:
            product_info.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[3]').text)
            driver.implicitly_wait(1)
        except:
            product_info.append(np.nan)
        # print(product_name)
        try: 
            review_text.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[2]/div').text)
            driver.implicitly_wait(1)
        except:
            review_text.append(np.nan)
        # print(review_content)
        try: 
            recommendation_count.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[2]/div/div/div/button/span').text)
            driver.implicitly_wait(1)
        except:
            recommendation_count.append(np.nan)
        try:
            driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[2]')
            image_yn.append('Y')
        except:
            image_yn.append('N')
            
    # time.sleep(random.uniform(1, 2))
    # click next button
    driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
    time.sleep(1)
    page += 1

driver.quit()
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

● 현재 페이지: 1 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 2 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 3 , 해당 페이지 리뷰수: 20


...

● 현재 페이지: 318 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

● 현재 페이지: 319 , 해당 페이지 리뷰수: 20


  0%|          | 0/20 [00:00<?, ?it/s]

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x0000000102a544dc chromedriver + 4162780
1   chromedriver                        0x0000000102a4c664 chromedriver + 4130404
2   chromedriver                        0x00000001026a39f8 chromedriver + 293368
3   chromedriver                        0x00000001026eb870 chromedriver + 587888
4   chromedriver                        0x00000001026df994 chromedriver + 539028
5   chromedriver                        0x00000001026df250 chromedriver + 537168
6   chromedriver                        0x0000000102729ab0 chromedriver + 842416
7   chromedriver                        0x00000001026dd6bc chromedriver + 530108
8   chromedriver                        0x00000001026de930 chromedriver + 534832
9   chromedriver                        0x0000000102a19e08 chromedriver + 3923464
10  chromedriver                        0x0000000102a1e3dc chromedri

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   review_date           60 non-null     object
 1   user_id               60 non-null     object
 2   rating                60 non-null     object
 3   product_info          60 non-null     object
 4   review_text           60 non-null     object
 5   image_yn              60 non-null     object
 6   recommendation_count  60 non-null     object
dtypes: object(7)
memory usage: 3.4+ KB


In [69]:
df.shape

(6380, 7)

In [71]:
df.to_excel("FootHealth.xlsx", index=False)

In [67]:
df.to_excel("FootHealth.xlsx", index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [66]:
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.02.,ccm5***,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 270~280...,아들이 군대갈때 평발이 있어서 가져갔는데 훈련받을때도 발이 아프지 않고 편하다고 하...,Y,0
1,24.01.02.,kore*****,5,교정 단계: 2단계 / 제품 모델: 액티런(DFH803) / 사이즈: 225~235...,평발이라 조금만 걸어도 발이 아파서 ㅠㅠ기능성 깔창을 계속 사용해온 1인 입니다 여...,N,0
2,24.01.01.,imju******,5,교정 단계: 2단계 / 제품 모델: 액티런(DFH803) / 사이즈: 270~280...,일때문에 자주 걷는편인데\n자꾸 아프게 되어 여러가지 알아보다\n구매했습니다\n일단...,N,0
3,23.12.31.,sept*********,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 225~235...,제가 진짜 좋다는 깔창은 나 사서 써본 사람으로써 이제품 찐으로 좋아요 ㅠㅠ \n하...,N,0
4,23.12.31.,dmsq********,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 210~220...,한달사용기몇년째 군화에 넣고 활용중인데 좋아요. 가격이 좀 비싸서 타회사 다른 상품...,N,0
